# Assigment 5

Welcome to the last ML assignment in which you will experiment (and hopefully get some experience) with a technique called Principal Component Analysis (PCA). This method is often used in Machine Learning for "dimensionality reduction", that is for reducing the dimensionality of the data. Put in other words, it is used to "compress" the data without (hopefully) lossing too much information. PCA can also be used for compressing images, during exploratory data analysis (when we try to "get a sense of the data") or as a visualization technique (when we try to "see what our data looks like"). 

We can also view PCA as an unsupervised learning algorithm that learns a representation of data. This learned representation is based on two criteria. First, PCA learns a representation that has lower dimensionality than the original data. Second, elements in this representation have no linear correlations with each other.

PCA learns orhogonal linear transformation of the data. This transformation tries to learn the best projection that keeps as much information about the data as possible, meassured by least-squared reconstruction error. Let us start with some neccessary imports, as we usually do.

**Note**: Due to time constraints, this assignment will_"scratch the surface"_  at best, when it comes to deriving the actual process of obtaining the principal components. If you would like a more in-depth guide, we strongly recommend [this tutorial](https://tkv.io/posts/tutorial-on-pca/).

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_olivetti_faces
from utils import plot_faces

In this first example we will work with two dimensional data so we can easily visualize the results. Here we prepared some highly correlated data (correlation coefficient at *0.9* -- you can play around with this [[literally, even in multiplayer mode](http://guessthecorrelation.com/)]). 

In [ ]:
xx = np.array([-0.51, 51.2])
yy = np.array([0.33, 51.6])
means = [xx.mean(), yy.mean()]  
stds = [xx.std() / 3, yy.std() / 3]
corr = 0.9 # correlation
covs = [[stds[0]**2, stds[0]*stds[1]*corr], 
        [stds[0]*stds[1]*corr, stds[1]**2]] 

X = np.random.multivariate_normal(means, covs, 100)
plt.scatter(X[:, 0], X[:, 1])
plt.show()

PCA is very sensitive to scale of the features. For PCA to work properly (that is in order for it to give us the results we described above) you *have to* subtract the mean from the data so every feature has **zero mean**.

Otherwise we risk that the first principal component may just resemble the mean.

In [ ]:
# Perform feature scaling here

Since PCA should project our data to the space where elements have no linear correlations, we can start by examining this relationship between the dimensons in our original data. We can get the unbiased sample covariance matrix associated with X by computing $$\Sigma = \frac{1}{n}X^{T}X$$ or by using numpy's `np.cov` function

In [ ]:
# Compute and store covariance matrix here.
sigma = None

*Please, briefly describe here what can we say about our data sample from the computed covariance matrix.*

Now, it is time to compute eigenvalues and eigenvectors of our data. As it turns out, these eigenvectors correspond to the principal components of our data. We can learn an n-dimensional representation that provides the best reconstruction of the original data (in the sense of mean squared error) and that this representation actually corresponds to the first n principal component of the data and therefore to our computed eigenvectors. Thus we can use PCA as a simple and eﬀective dimensionality reduction method that preserves as much of the information in the data as possible (again, as measured by least-squares reconstruction error).

Since our matrix $\Sigma$ is positive semi-definite, we can always find eigenvectors.

Probably the easiest way of doing so is by using `numpy`'s `np.linalg.eigh` function which returns two matricies $W$ and $V$. Matrix $W$ contains eigenvalues (in sorted order) and matrix $V$ contains eigenvectors.

For PCA to work, these vectors have to be of unit length. Luckily most libraries already takes care of normalizing these vectors. In the next cell, please use this function to extract eigenvectors with coresponding eigenvalues.

In [ ]:
# W, V = None

As the last step of PCA we have to transform our original data using the learned transformation.

We now have all principal components of our data. If we want to transform our data to $k$ dimensions, we can extract $k$ (in our toy 2-dimensional case just one) of these components (eigenvectors) and construct a matrix $U_k$ of $n \times k$ dimensions. This will be our linear transformation matrix. With this matrix we transform our original data $X$ of shape $ n \times m$, where $n$ is the number of examples and $m$ is the number of original features, by simply performing $$X_{transformed} = X \cdot U_k^{T}$$

In [ ]:
# Extract and transform over here.
X_new = None

Now we can use a "stock" PCA implementation from `sklearn` library to compare our vectors. The difference between the two transformed vectors should be very small.

In [ ]:
pca = PCA(n_components=1)
X_tranformed = pca.fit_transform(X)
print("Diff: {}".format(np.sum(X_new - X_tranformed)))

----------------------------------------------------------------------------------------------------

For our second example we loaded a 20 dimensional data sample. 

PCA can also be viewed as a transformation that tries to keep as much variance in the data as possible while lowering its dimension. This is due to the fact that we are keeping the $k$ largest principal components. The first principal component tries to account for as much of the variability of the data as possible. The second one tries to account for as much of the remaining variability, and so on. By keeping the $k$ largest principal components we try to keep as much variance as possible (within those $k$ dimensions).

We can also see how much variance each dimension retains. The fitted `PCA` object contains variables `explained_variance_ ` and `explained_variance_ratio_ `, which can help us answer this question. This is often used to help us decide how many principal components we will keep to retain as much variance as possible.

In the next cell please fit the PCA on provided data. What can we say about our data and its variance? How many components do we need to retain 99% of variance?

In [ ]:
data = np.loadtxt('data.csv', delimiter=',')

*Please, write your answer here.*

As we can see from the results, some of the dimensions have almost no added information value. This means that if we wanted to use this data in a classification or prediction setup, we can just as well only use half of our features. This will make our data smaller and therefore speed up our training and may also improve our prediction results since our models do not have to learn to 'ignore' these features.

---------------------------------------------------------------------------------------------------
One of the most famous uses of PCA is called **eigenfaces**. The Eigenface approach began with a search for a low-dimensional representation of face images. Principal component analysis could be used on a collection of face images to provide a set of basis features. These basis images (also known  as Eigenpictures) could be linearly combined to reconstruct images in the original training set.

Informaly, human face can be thought of as set of 'standardized features' that can be extracted from large set of faces by statistical analysis. Each face is a linear combination of these basic features (somewhat standardized faces). It can contain 10% of eigenface 1, 3% of eigenface 2 and so on...

To generate eigenfaces we loaded a standard face dataset which consists out of 400 faces. Each image is of size $64 \times 64$. To calculate PCA you first need to normalize each face. You sould compute average image and subtract it from each face.

Calculate the eigenvectors and eigenvalues. Each eigenvector has the same dimensionality (number of components) as the original image, and thus can seen as an image in and of itself. The eigenvectors are therefore called `eigenfaces`. They are the directions in which the images differ from the mean image.

In the next cell please normalize the loaded images of faces and compute the eigenfaces using sklearn's PCA method. You can use the provided `plot_faces(images, n_cols, n_rows)` function, where the first argument is a list of images of faces of length `n_cols * n_rows`. Values `n_cols` and `n_rows` are used to draw subplots, so if we want to visualize $k=6$ eigenfaces we would set something like `n_cols=3` and `n_rows=2`.

In [ ]:
# Load faces data
dataset = fetch_olivetti_faces(shuffle=True)
faces = dataset.data

n_rows, n_cols = 2, 3
n_components = n_rows * n_cols
n_samples, n_features = faces.shape

print("Dataset consists of {} faces and {} features".format(n_samples, n_features))

# normalize faces data
# faces = 

# plot sample data
plot_faces(faces[:n_components], n_cols, n_rows)
plt.show()

# fit the PCA

# show standardize faces
plot_faces(pca.components_, n_cols, n_rows)
plt.show()

# For (a ton of) bonus points

`sklearn`'s PCA objects also contain a function `inverse_transform()` which can transform the data back to its original space. As you can imagine, this can be used to reconstruct the face back to its original form. So for example, we can take a face, transform it to our $k$ eigenface space, change some of its values and transform it back to obtain a new face, or just try to reconstruct back the original face with $k$ principal components.

For bonus points, create an `pyplot` animation where in the $k$th-step of animation you will reconstruct original image with first $k$ principal components. We believe that [this matplotlib example](https://matplotlib.org/2.0.0/examples/animation/basic_example.html) may be of some help with regards to the animation.